In [226]:
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
import xgboost as xgb

xgb.set_config(verbosity=0)
import joblib
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [227]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/bandai_l2p_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/bandai_l2a_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/bandai_l2d_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/S_BANDAI_5frame.pkl') 

In [228]:
dataset = pd.read_csv('datasets/BANDAI_5frame.csv')
dataset.head()

,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude,EMOTION_P,EMOTION_A,EMOTION_D
0,0.433596,0.130036,0.335475,0.538619,0.184473,0.310089,0.210679,0.093915,0.236151,0.223894,...,-0.137483,-0.139467,0.092920,0.196129,0.221155,0.137483,0.139467,0.05,-0.05,0.0
1,0.348051,0.145184,0.256784,0.455501,0.197954,0.284215,0.172405,0.104876,0.236151,0.223894,...,-0.072441,-0.095084,0.048935,0.032804,0.070625,0.072096,0.045111,0.05,-0.05,0.0
2,0.320294,0.206306,0.180224,0.380190,0.234406,0.259905,0.133006,0.127385,0.236151,0.223894,...,-0.039016,-0.093423,0.030819,0.043059,0.028182,0.036532,0.014232,0.05,-0.05,0.0
3,0.400389,0.287300,0.136974,0.319861,0.279208,0.239450,0.112632,0.156856,0.236151,0.223894,...,-0.036825,-0.101489,0.044534,0.054382,0.015679,0.012813,0.021506,0.05,-0.05,0.0
4,0.465923,0.356129,0.136171,0.309995,0.313229,0.224432,0.112950,0.186680,0.236151,0.223894,...,-0.054338,-0.117573,0.044670,0.036618,0.020074,0.022012,0.024961,0.05,-0.05,0.0


In [229]:
train_dataset = dataset.sample(frac=0.8)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 62841
No Test Samples: 15710


In [230]:
train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
train_emotions_OG = train_emotions.copy()

test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions_OG = test_emotions.copy()

In [231]:
scaled_train = scaler.transform(train_dataset.copy())

train_emotions_p = model_p.predict(scaled_train)
train_emotions_a = model_a.predict(scaled_train)
train_emotions_d = model_d.predict(scaled_train)

rows = []
for i in range(len(train_dataset)):
    rows.append([train_emotions_p[i], train_emotions_a[i], train_emotions_d[i]])

train_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

train_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.099475,-0.752318,-0.252752
1,0.100070,-0.750083,-0.252383
2,0.600116,0.402414,0.101721
3,0.077142,0.605400,0.315327
4,0.100015,0.596519,0.397691


In [232]:
train_emotions_OG.head()

,EMOTION_P,EMOTION_A,EMOTION_D
49898,0.1,-0.75,-0.25
39312,0.1,-0.75,-0.25
23511,0.6,0.40,0.10
17829,0.1,0.60,0.40
62663,0.1,0.60,0.40


In [233]:
# Remove LMA features whose predictions are too different from the real one
index_for_removal = []
for ind in train_emotions.index:
    predicted = np.asarray([train_emotions.iloc[ind]])
    true = np.asarray([train_emotions_OG.iloc[ind]])
    
    #if(true[0][0] == 0.05 and true[0][1] == -0.05 and true[0][2] == 0.0):
    #    index_for_removal.append(ind)
    #    continue
    
    #print(predicted)
    #print(true)
    
    mae_errors = mean_absolute_error(true, predicted, multioutput='raw_values')
    #print(mae_errors)
    
    if(mae_errors[0] > 0.2 or mae_errors[1] > 0.2 or mae_errors[2] > 0.2):
        index_for_removal.append(ind)
        continue
    
    mae_error = mean_absolute_error(true, predicted)
    #print(mae_error)
    
    if(mae_error > 0.03):
        index_for_removal.append(ind)
        continue
    
    #print()

print(index_for_removal)


[3, 7, 18, 29, 41, 53, 62, 66, 78, 80, 131, 150, 161, 173, 177, 194, 195, 197, 262, 275, 288, 291, 294, 300, 326, 341, 373, 388, 391, 405, 408, 414, 433, 436, 441, 458, 459, 527, 549, 550, 569, 571, 572, 596, 616, 617, 626, 642, 671, 674, 692, 693, 704, 705, 729, 732, 744, 745, 753, 780, 788, 801, 810, 824, 829, 849, 850, 854, 855, 876, 883, 886, 893, 937, 939, 944, 952, 975, 984, 1022, 1039, 1059, 1078, 1082, 1085, 1087, 1106, 1117, 1130, 1133, 1137, 1153, 1172, 1175, 1212, 1232, 1263, 1276, 1280, 1296, 1307, 1316, 1331, 1373, 1376, 1381, 1395, 1413, 1424, 1449, 1451, 1457, 1460, 1462, 1466, 1480, 1518, 1519, 1527, 1546, 1548, 1556, 1557, 1563, 1579, 1585, 1600, 1604, 1620, 1630, 1635, 1640, 1658, 1666, 1667, 1675, 1683, 1700, 1718, 1725, 1728, 1753, 1754, 1760, 1774, 1791, 1792, 1823, 1830, 1841, 1844, 1859, 1865, 1883, 1885, 1887, 1893, 1899, 1908, 1912, 1918, 1930, 1934, 1984, 1987, 1997, 1998, 2015, 2027, 2038, 2062, 2075, 2098, 2105, 2116, 2119, 2144, 2150, 2168, 2169, 2190, 2199

In [234]:
train_emotions_OG.drop(train_emotions_OG.index[index_for_removal], inplace=True)
print(train_emotions_OG.shape)
train_emotions_OG.head()

(58229, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
49898,0.1,-0.75,-0.25
39312,0.1,-0.75,-0.25
23511,0.6,0.40,0.10
62663,0.1,0.60,0.40
3986,0.6,0.40,0.10


In [235]:
train_emotions.drop(train_emotions.index[index_for_removal], inplace=True)
print(train_emotions.shape)
train_emotions.head()

(58229, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
0,0.099475,-0.752318,-0.252752
1,0.100070,-0.750083,-0.252383
2,0.600116,0.402414,0.101721
4,0.100015,0.596519,0.397691
5,0.599353,0.401356,0.096966


In [236]:
train_dataset.drop(train_dataset.index[index_for_removal], inplace=True)
print(train_dataset.shape)
train_dataset.head()

(58229, 25)


,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
49898,0.796981,0.389229,0.436204,0.373207,0.460221,0.410692,0.289226,0.277202,0.236151,0.223894,...,-0.087921,-0.056894,-0.034818,-0.086239,-0.061217,0.010573,0.012621,0.009331,0.014102,0.009896
39312,0.903563,0.383400,0.534512,0.574015,0.446864,0.495521,0.283210,0.347510,0.236151,0.223894,...,-0.061881,-0.074722,-0.040788,-0.045168,-0.074704,0.014710,0.010490,0.018722,0.016683,0.004604
23511,0.605146,0.277083,0.333470,0.648512,0.320145,0.323761,0.172499,0.125225,0.236151,0.223894,...,-0.074089,-0.127328,-0.129829,-0.108098,-0.098851,0.009330,0.014946,0.028910,0.008939,0.005300
62663,0.503210,0.391723,0.384899,0.570763,0.375746,0.200700,0.374048,0.237124,0.236151,0.223894,...,-0.134598,-0.129042,-0.076314,-0.093803,-0.103426,0.024912,0.022874,0.009811,0.028791,0.007109
3986,0.957899,0.535175,0.452014,0.379482,0.473959,0.481866,0.325451,0.304671,0.236151,0.223894,...,-0.083324,-0.038124,-0.032183,-0.118196,-0.066099,0.018335,0.013545,0.008846,0.005404,0.010136


In [237]:
scaled_test = scaler.transform(test_dataset.copy())

test_emotions_p = model_p.predict(scaled_test)
test_emotions_a = model_a.predict(scaled_test)
test_emotions_d = model_d.predict(scaled_test)

rows = []
for i in range(len(test_dataset)):
    rows.append([test_emotions_p[i], test_emotions_a[i], test_emotions_d[i]])

test_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

test_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.051494,-0.039454,0.003107
1,-0.100236,-0.596791,-0.154467
2,0.299907,0.398420,0.595462
3,0.100203,0.604365,0.402592
4,0.095544,0.455239,0.370865


In [238]:
test_emotions_OG.head()

,EMOTION_P,EMOTION_A,EMOTION_D
37419,0.05,-0.05,0.00
56226,-0.10,-0.60,-0.15
65859,0.30,0.40,0.60
19700,0.10,0.60,0.40
49733,0.10,0.60,0.40


In [239]:
# Remove LMA features whose predictions are too different from the real one
index_for_removal = []
for ind in test_emotions.index:
    predicted = np.asarray([test_emotions.iloc[ind]])
    true = np.asarray([test_emotions_OG.iloc[ind]])
    
    #print(predicted)
    #print(true)
    
    mae_errors = mean_absolute_error(true, predicted, multioutput='raw_values')
    #print(mae_errors)
    
    if(mae_errors[0] > 0.2 or mae_errors[1] > 0.2 or mae_errors[2] > 0.2):
        index_for_removal.append(ind)
        continue
    
    mae_error = mean_absolute_error(true, predicted)
    #print(mae_error)
    
    if(mae_error > 0.03):
        index_for_removal.append(ind)
        continue
    
    #print()

print(index_for_removal)


[4, 6, 24, 27, 40, 55, 58, 65, 82, 101, 134, 158, 164, 179, 200, 220, 243, 244, 278, 296, 313, 317, 325, 343, 364, 380, 385, 394, 424, 433, 462, 466, 473, 474, 496, 506, 508, 512, 527, 528, 538, 559, 563, 566, 578, 620, 623, 634, 639, 662, 694, 715, 721, 747, 751, 755, 759, 780, 793, 802, 806, 819, 822, 826, 832, 840, 854, 862, 911, 933, 937, 938, 958, 974, 982, 983, 988, 1002, 1023, 1042, 1056, 1066, 1078, 1094, 1096, 1100, 1104, 1112, 1114, 1151, 1182, 1188, 1205, 1206, 1222, 1227, 1269, 1272, 1277, 1287, 1290, 1292, 1310, 1319, 1341, 1342, 1348, 1350, 1352, 1373, 1388, 1393, 1448, 1471, 1479, 1516, 1559, 1570, 1571, 1596, 1613, 1622, 1627, 1635, 1665, 1677, 1716, 1745, 1763, 1769, 1800, 1820, 1844, 1851, 1873, 1875, 1877, 1879, 1883, 1898, 1900, 1901, 1903, 1919, 1925, 1928, 1949, 1974, 1999, 2001, 2008, 2018, 2024, 2025, 2076, 2087, 2096, 2097, 2118, 2120, 2125, 2131, 2138, 2141, 2150, 2181, 2199, 2225, 2226, 2234, 2292, 2309, 2319, 2349, 2356, 2364, 2378, 2387, 2419, 2452, 2470, 2

In [240]:
test_emotions_OG.drop(test_emotions_OG.index[index_for_removal], inplace=True)
print(test_emotions_OG.shape)
test_emotions_OG.head()

(14564, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
37419,0.05,-0.05,0.00
56226,-0.10,-0.60,-0.15
65859,0.30,0.40,0.60
19700,0.10,0.60,0.40
44449,0.05,-0.05,0.00


In [241]:
test_emotions.drop(test_emotions.index[index_for_removal], inplace=True)
print(test_emotions.shape)
test_emotions.head()

(14564, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
0,0.051494,-0.039454,0.003107
1,-0.100236,-0.596791,-0.154467
2,0.299907,0.398420,0.595462
3,0.100203,0.604365,0.402592
5,0.050174,-0.045575,0.002002


In [242]:
test_dataset.drop(test_dataset.index[index_for_removal], inplace=True)
print(test_dataset.shape)
test_dataset.head()

(14564, 25)


,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
37419,0.464690,0.095188,0.357215,0.628172,0.276610,0.372085,0.123448,0.150418,0.236151,0.223894,...,-0.067973,-0.095992,-0.099821,-0.114918,-0.078886,0.009023,0.013269,0.017109,0.018505,0.007355
56226,0.574909,0.307601,0.272455,0.410620,0.380857,0.378913,0.271549,0.269376,0.236151,0.223894,...,-0.051414,-0.036004,-0.025033,-0.065891,-0.045062,0.004328,0.009201,0.007693,0.015086,0.006038
65859,0.740076,0.409941,0.340451,0.295422,0.413895,0.422891,0.320570,0.291677,0.236151,0.223894,...,-0.178694,-0.120949,-0.095157,-0.196285,-0.143627,0.036501,0.035673,0.029382,0.005784,0.025115
19700,0.599736,0.460908,0.340772,0.867680,0.347605,0.292026,0.094260,0.344792,0.236151,0.223894,...,-0.188515,-0.190831,-0.279597,-0.174220,-0.195540,0.026340,0.019924,0.007631,0.028016,0.017682
44449,0.701045,0.425058,0.285326,0.403893,0.425931,0.394380,0.290389,0.241355,0.236151,0.223894,...,-0.059008,-0.035053,-0.018422,-0.101375,-0.054058,0.006474,0.007975,0.003790,0.008920,0.004815



# AutoEncoder


In [243]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [244]:
"""
self.encoder = tf.keras.Sequential([
            layers.Dense(20, activation='relu'),
            layers.Dense(15, activation='linear'),
            layers.Dense(10, activation='relu'),
            layers.Dense(latent_dim, activation='relu'),
        ])
        
        self.decoder = tf.keras.Sequential([
            layers.Dense(10, activation='relu'),
            layers.Dense(15, activation='linear'),
            layers.Dense(20, activation='relu'),
            layers.Dense(25, activation='linear'),
        ])
epochs = 128
"""

latent_dim = 5

class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        
        self.encoder = tf.keras.Sequential([
            layers.Dense(20, activation='relu'),
            layers.Dense(15, activation='linear'),
            layers.Dense(10, activation='relu'),
            layers.Dense(latent_dim, activation='linear'),
        ])
        
        self.decoder = tf.keras.Sequential([
            layers.Dense(10, activation='relu'),
            layers.Dense(15, activation='linear'),
            layers.Dense(20, activation='relu'),
            layers.Dense(25, activation='linear'),
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(latent_dim)

## Train

In [245]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [246]:
autoencoder.fit(train_dataset, train_dataset,
                #epochs=4096,
                epochs=512,
                shuffle=True,
                validation_data=(test_dataset, test_dataset), verbose=2)

Epoch 1/512
1820/1820 - 3s - loss: 0.0035 - val_loss: 0.0022 - 3s/epoch - 2ms/step
Epoch 2/512
1820/1820 - 2s - loss: 0.0022 - val_loss: 0.0022 - 2s/epoch - 1ms/step
Epoch 3/512
1820/1820 - 2s - loss: 0.0022 - val_loss: 0.0022 - 2s/epoch - 1ms/step
Epoch 4/512
1820/1820 - 2s - loss: 0.0022 - val_loss: 0.0022 - 2s/epoch - 1ms/step
Epoch 5/512
1820/1820 - 2s - loss: 0.0022 - val_loss: 0.0022 - 2s/epoch - 1ms/step
Epoch 6/512
1820/1820 - 2s - loss: 0.0022 - val_loss: 0.0022 - 2s/epoch - 1ms/step
Epoch 7/512
1820/1820 - 2s - loss: 0.0016 - val_loss: 0.0015 - 2s/epoch - 1ms/step
Epoch 8/512
1820/1820 - 2s - loss: 0.0011 - val_loss: 0.0010 - 2s/epoch - 1ms/step
Epoch 9/512
1820/1820 - 2s - loss: 9.8995e-04 - val_loss: 9.9288e-04 - 2s/epoch - 1ms/step
Epoch 10/512
1820/1820 - 2s - loss: 9.7924e-04 - val_loss: 9.8671e-04 - 2s/epoch - 1ms/step
Epoch 11/512
1820/1820 - 2s - loss: 9.7100e-04 - val_loss: 9.9370e-04 - 2s/epoch - 1ms/step
Epoch 12/512
1820/1820 - 2s - loss: 9.6252e-04 - val_loss: 9.

Epoch 140/512
1820/1820 - 2s - loss: 4.4872e-04 - val_loss: 4.6347e-04 - 2s/epoch - 1ms/step
Epoch 141/512
1820/1820 - 2s - loss: 4.5102e-04 - val_loss: 4.6753e-04 - 2s/epoch - 1ms/step
Epoch 142/512
1820/1820 - 2s - loss: 4.4686e-04 - val_loss: 4.5428e-04 - 2s/epoch - 1ms/step
Epoch 143/512
1820/1820 - 2s - loss: 4.4953e-04 - val_loss: 4.6529e-04 - 2s/epoch - 1ms/step
Epoch 144/512
1820/1820 - 2s - loss: 4.4948e-04 - val_loss: 4.4700e-04 - 2s/epoch - 1ms/step
Epoch 145/512
1820/1820 - 2s - loss: 4.4933e-04 - val_loss: 4.5970e-04 - 2s/epoch - 1ms/step
Epoch 146/512
1820/1820 - 2s - loss: 4.4656e-04 - val_loss: 4.5800e-04 - 2s/epoch - 1ms/step
Epoch 147/512
1820/1820 - 2s - loss: 4.4858e-04 - val_loss: 4.5436e-04 - 2s/epoch - 1ms/step
Epoch 148/512
1820/1820 - 2s - loss: 4.5003e-04 - val_loss: 4.4738e-04 - 2s/epoch - 1ms/step
Epoch 149/512
1820/1820 - 2s - loss: 4.4782e-04 - val_loss: 4.5355e-04 - 2s/epoch - 1ms/step
Epoch 150/512
1820/1820 - 2s - loss: 4.4738e-04 - val_loss: 4.5277e-04

1820/1820 - 2s - loss: 3.8892e-04 - val_loss: 3.8778e-04 - 2s/epoch - 1ms/step
Epoch 229/512
1820/1820 - 2s - loss: 3.8708e-04 - val_loss: 4.0178e-04 - 2s/epoch - 1ms/step
Epoch 230/512
1820/1820 - 2s - loss: 3.8719e-04 - val_loss: 3.9104e-04 - 2s/epoch - 1ms/step
Epoch 231/512
1820/1820 - 2s - loss: 3.8757e-04 - val_loss: 3.9617e-04 - 2s/epoch - 1ms/step
Epoch 232/512
1820/1820 - 2s - loss: 3.8628e-04 - val_loss: 3.8914e-04 - 2s/epoch - 1ms/step
Epoch 233/512
1820/1820 - 2s - loss: 3.8752e-04 - val_loss: 4.1033e-04 - 2s/epoch - 1ms/step
Epoch 234/512
1820/1820 - 2s - loss: 3.8709e-04 - val_loss: 4.1009e-04 - 2s/epoch - 1ms/step
Epoch 235/512
1820/1820 - 2s - loss: 3.8507e-04 - val_loss: 3.8626e-04 - 2s/epoch - 1ms/step
Epoch 236/512
1820/1820 - 2s - loss: 3.8519e-04 - val_loss: 3.9058e-04 - 2s/epoch - 1ms/step
Epoch 237/512
1820/1820 - 2s - loss: 3.8608e-04 - val_loss: 3.8497e-04 - 2s/epoch - 1ms/step
Epoch 238/512
1820/1820 - 2s - loss: 3.8285e-04 - val_loss: 3.8955e-04 - 2s/epoch - 

Epoch 317/512
1820/1820 - 2s - loss: 3.7552e-04 - val_loss: 3.8440e-04 - 2s/epoch - 1ms/step
Epoch 318/512
1820/1820 - 2s - loss: 3.7596e-04 - val_loss: 3.8722e-04 - 2s/epoch - 1ms/step
Epoch 319/512
1820/1820 - 2s - loss: 3.7612e-04 - val_loss: 3.7880e-04 - 2s/epoch - 1ms/step
Epoch 320/512
1820/1820 - 2s - loss: 3.7937e-04 - val_loss: 3.7513e-04 - 2s/epoch - 1ms/step
Epoch 321/512
1820/1820 - 2s - loss: 3.7416e-04 - val_loss: 3.7810e-04 - 2s/epoch - 1ms/step
Epoch 322/512
1820/1820 - 2s - loss: 3.7557e-04 - val_loss: 3.8138e-04 - 2s/epoch - 1ms/step
Epoch 323/512
1820/1820 - 2s - loss: 3.7470e-04 - val_loss: 3.7857e-04 - 2s/epoch - 1ms/step
Epoch 324/512
1820/1820 - 2s - loss: 3.7574e-04 - val_loss: 3.7550e-04 - 2s/epoch - 1ms/step
Epoch 325/512
1820/1820 - 2s - loss: 3.7489e-04 - val_loss: 3.7985e-04 - 2s/epoch - 1ms/step
Epoch 326/512
1820/1820 - 2s - loss: 3.7472e-04 - val_loss: 3.7683e-04 - 2s/epoch - 1ms/step
Epoch 327/512
1820/1820 - 2s - loss: 3.7702e-04 - val_loss: 3.7391e-04

1820/1820 - 2s - loss: 3.7245e-04 - val_loss: 3.8244e-04 - 2s/epoch - 1ms/step
Epoch 406/512
1820/1820 - 2s - loss: 3.6970e-04 - val_loss: 4.2414e-04 - 2s/epoch - 1ms/step
Epoch 407/512
1820/1820 - 2s - loss: 3.6905e-04 - val_loss: 3.7659e-04 - 2s/epoch - 1ms/step
Epoch 408/512
1820/1820 - 2s - loss: 3.7143e-04 - val_loss: 3.7599e-04 - 2s/epoch - 1ms/step
Epoch 409/512
1820/1820 - 2s - loss: 3.7023e-04 - val_loss: 3.8364e-04 - 2s/epoch - 1ms/step
Epoch 410/512
1820/1820 - 2s - loss: 3.7094e-04 - val_loss: 3.7842e-04 - 2s/epoch - 1ms/step
Epoch 411/512
1820/1820 - 2s - loss: 3.7007e-04 - val_loss: 3.7207e-04 - 2s/epoch - 1ms/step
Epoch 412/512
1820/1820 - 2s - loss: 3.6919e-04 - val_loss: 3.6850e-04 - 2s/epoch - 1ms/step
Epoch 413/512
1820/1820 - 2s - loss: 3.7008e-04 - val_loss: 3.8075e-04 - 2s/epoch - 1ms/step
Epoch 414/512
1820/1820 - 2s - loss: 3.7024e-04 - val_loss: 3.7471e-04 - 2s/epoch - 1ms/step
Epoch 415/512
1820/1820 - 2s - loss: 3.7017e-04 - val_loss: 3.7204e-04 - 2s/epoch - 

Epoch 494/512
1820/1820 - 2s - loss: 3.6701e-04 - val_loss: 3.6726e-04 - 2s/epoch - 1ms/step
Epoch 495/512
1820/1820 - 2s - loss: 3.6601e-04 - val_loss: 3.6751e-04 - 2s/epoch - 1ms/step
Epoch 496/512
1820/1820 - 2s - loss: 3.6644e-04 - val_loss: 3.7106e-04 - 2s/epoch - 1ms/step
Epoch 497/512
1820/1820 - 2s - loss: 3.6682e-04 - val_loss: 3.8317e-04 - 2s/epoch - 1ms/step
Epoch 498/512
1820/1820 - 2s - loss: 3.6510e-04 - val_loss: 3.6737e-04 - 2s/epoch - 1ms/step
Epoch 499/512
1820/1820 - 2s - loss: 3.6879e-04 - val_loss: 3.7338e-04 - 2s/epoch - 1ms/step
Epoch 500/512
1820/1820 - 2s - loss: 3.6538e-04 - val_loss: 3.7267e-04 - 2s/epoch - 1ms/step
Epoch 501/512
1820/1820 - 2s - loss: 3.6793e-04 - val_loss: 3.6438e-04 - 2s/epoch - 1ms/step
Epoch 502/512
1820/1820 - 2s - loss: 3.6671e-04 - val_loss: 3.7067e-04 - 2s/epoch - 1ms/step
Epoch 503/512
1820/1820 - 2s - loss: 3.6669e-04 - val_loss: 3.6854e-04 - 2s/epoch - 1ms/step
Epoch 504/512
1820/1820 - 2s - loss: 3.6695e-04 - val_loss: 3.7024e-04

In [247]:
index = 0

sample = np.asarray(test_dataset.iloc[index])
sample = sample.reshape(1,-1)
print(sample)

actual_coords = np.asarray(test_emotions_OG.iloc[index])

[[ 0.46468982  0.09518754  0.35721504  0.62817162  0.27660996  0.37208495
   0.12344829  0.15041816  0.23615099  0.22389401  0.26728641  0.07792138
   0.13571444  0.09965176  0.23814863 -0.06797334 -0.0959922  -0.09982077
  -0.11491793 -0.07888636  0.00902275  0.01326883  0.01710929  0.01850493
   0.00735474]]


In [248]:
generated = autoencoder.encoder.predict(sample)
print(generated)

[[ 0.7410891  -0.8514057   0.67808604  0.38241822 -0.6003839 ]]


In [249]:
regen = autoencoder.decoder.predict(generated)
print(regen)

[[ 0.4711386   0.10781887  0.3648116   0.6263461   0.23296012  0.36315882
   0.15305778  0.14072284  0.23649606  0.22392355  0.27909455  0.08938954
   0.11657986  0.08562563  0.2217633  -0.0622624  -0.09870583 -0.10744667
  -0.08290666 -0.07861131  0.01568681  0.01451125  0.02532215  0.02003278
   0.00915953]]


In [250]:
scaled_sample = scaler.transform(sample)

real_coordinates = (
    model_p.predict(scaled_sample),
    model_a.predict(scaled_sample),
    model_d.predict(scaled_sample)
)

scaled_regen = scaler.transform(regen)

generated_coordinates = (
    model_p.predict(scaled_regen),
    model_a.predict(scaled_regen),
    model_d.predict(scaled_regen)
)


print('Real: %s' % np.asarray(actual_coords))
print('Predicted: %s' % np.asarray(real_coordinates).transpose()[0])
print('Generated: %s' % np.asarray(generated_coordinates).transpose()[0])

Real: [ 0.05 -0.05  0.  ]
Predicted: [ 0.05149397 -0.03945355  0.00310656]
Generated: [0.20675895 0.27070683 0.44560018]


### Test

In [251]:
# 2nd - Use Decoder to get LMA features
generated_lma = autoencoder.decoder.predict(autoencoder.encoder.predict(test_dataset))
print(generated_lma.shape)

scaled_generated_lma = scaler.transform(generated_lma)

(14564, 25)


In [252]:
# 3rd - Get Predicted Emotions
gen_emotions_p = model_p.predict(scaled_generated_lma)
gen_emotions_a = model_a.predict(scaled_generated_lma)
gen_emotions_d = model_d.predict(scaled_generated_lma)

rows = []
for i in range(len(scaled_generated_lma)):
    rows.append([gen_emotions_p[i], gen_emotions_a[i], gen_emotions_d[i]])

gen_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

gen_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.206759,0.270707,0.445600
1,-0.067457,-0.482045,-0.122257
2,0.298849,0.364900,0.369557
3,0.268155,0.537362,0.321638
4,0.058995,-0.546982,-0.141038


In [253]:
# 4th - Get MAE
import random

mae_errors = mean_absolute_error(test_emotions, gen_emotions, multioutput='raw_values')
mse_errors = mean_squared_error(test_emotions, gen_emotions, multioutput='raw_values')

features = ["PLEASURE", "AROUSAL", "DOMINANCE"
         ]

print("Overall MAE: " + str(mean_absolute_error(test_emotions, gen_emotions)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print()
    
for i in range(30):
    row = random.randint(0, len(test_emotions))

    print("Real: " + str([test_emotions_OG.iloc[row,0], test_emotions_OG.iloc[row,1], test_emotions_OG.iloc[row,2]]))
    print("Predicted: " + str([test_emotions.iloc[row,0], test_emotions.iloc[row,1], test_emotions.iloc[row,2]]))
    print("Generated: " + str([gen_emotions.iloc[row,0], gen_emotions.iloc[row,1], gen_emotions.iloc[row,2]]))
    print()

Overall MAE: 0.17245407

==PLEASURE==
MSE: 0.02867
MAE: 0.10913

==AROUSAL==
MSE: 0.11923
MAE: 0.24699

==DOMINANCE==
MSE: 0.05323
MAE: 0.16125

Real: [-0.6, 0.7, -0.8]
Predicted: [-0.5998465, 0.6992476, -0.7925069]
Generated: [-0.49843955, 0.58502334, -0.7456362]

Real: [0.4, 0.2, 0.35]
Predicted: [0.40018123, 0.20338212, 0.35754588]
Generated: [0.085594036, 0.08376298, 0.18642282]

Real: [-0.1, -0.6, -0.15]
Predicted: [-0.099480234, -0.59608454, -0.15310198]
Generated: [-0.05111861, -0.5278755, -0.13003989]

Real: [0.1, -0.75, -0.25]
Predicted: [0.09921702, -0.74691474, -0.25073567]
Generated: [0.21060407, 0.11803431, 0.28973988]

Real: [0.3, 0.4, 0.6]
Predicted: [0.2995398, 0.40310386, 0.60147655]
Generated: [0.23384021, 0.15567097, 0.30589303]

Real: [-0.1, -0.6, -0.15]
Predicted: [-0.09954496, -0.60518265, -0.14653772]
Generated: [-0.10676964, -0.5749768, -0.16425763]

Real: [0.6, 0.4, 0.1]
Predicted: [0.5996891, 0.39918226, 0.09773906]
Generated: [0.3498993, 0.39441755, 0.1022104

In [ ]:
# error on 4096 was 0.16

## Save Model

In [282]:
autoencoder.save("models/bandai/5frame/ae/autoencoder_5")

INFO:tensorflow:Assets written to: models/bandai/5frame/ae/autoencoder_5/assets



# PAD - Latent Space 

In [127]:
autoencoder = keras.models.load_model('models/bandai/5frame/ae/autoencoder_5')

In [254]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

In [255]:
generated_train = autoencoder.encoder.predict(train_dataset)
generated_test = autoencoder.encoder.predict(test_dataset)

generated_train_x = train_emotions.copy()
generated_test_x = test_emotions.copy()
#generated_train_x = train_emotions_OG.copy()
#generated_test_x = test_emotions_OG.copy()

In [256]:
# Remove LMA features whose predictions are too different from the real one
generated_decoded_train = autoencoder.decoder.predict(generated_train)

index_for_removal = []

for ind in range(0,generated_decoded_train.shape[0]):
    print(str(ind) + " / " + str(generated_decoded_train.shape[0]))
    
    predicted = np.asarray([generated_decoded_train[ind]])
    true = np.asarray([train_emotions.iloc[ind]])

    predicted = scaler.transform(predicted)

    predicted = (
        model_p.predict(predicted),
        model_a.predict(predicted),
        model_d.predict(predicted)
    )
    
    predicted = np.asarray(predicted).transpose()
    
    #print(predicted)
    #print(true)
    
    mae_errors = mean_absolute_error(true, predicted, multioutput='raw_values')
    #print(mae_errors)
    
    if(mae_errors[0] > 0.25 or mae_errors[1] > 0.25 or mae_errors[2] > 0.25):
        index_for_removal.append(ind)
        continue
    
    mae_error = mean_absolute_error(true, predicted)
    #print(mae_error)
    
    if(mae_error > 0.15):
        index_for_removal.append(ind)
        continue
    
    #print()

print(len(index_for_removal))


0 / 58229
1 / 58229
2 / 58229
3 / 58229
4 / 58229
5 / 58229
6 / 58229
7 / 58229
8 / 58229
9 / 58229
10 / 58229
11 / 58229
12 / 58229
13 / 58229
14 / 58229
15 / 58229
16 / 58229
17 / 58229
18 / 58229
19 / 58229
20 / 58229
21 / 58229
22 / 58229
23 / 58229
24 / 58229
25 / 58229
26 / 58229
27 / 58229
28 / 58229
29 / 58229
30 / 58229
31 / 58229
32 / 58229
33 / 58229
34 / 58229
35 / 58229
36 / 58229
37 / 58229
38 / 58229
39 / 58229
40 / 58229
41 / 58229
42 / 58229
43 / 58229
44 / 58229
45 / 58229
46 / 58229
47 / 58229
48 / 58229
49 / 58229
50 / 58229
51 / 58229
52 / 58229
53 / 58229
54 / 58229
55 / 58229
56 / 58229
57 / 58229
58 / 58229
59 / 58229
60 / 58229
61 / 58229
62 / 58229
63 / 58229
64 / 58229
65 / 58229
66 / 58229
67 / 58229
68 / 58229
69 / 58229
70 / 58229
71 / 58229
72 / 58229
73 / 58229
74 / 58229
75 / 58229
76 / 58229
77 / 58229
78 / 58229
79 / 58229
80 / 58229
81 / 58229
82 / 58229
83 / 58229
84 / 58229
85 / 58229
86 / 58229
87 / 58229
88 / 58229
89 / 58229
90 / 58229
91 / 5822

704 / 58229
705 / 58229
706 / 58229
707 / 58229
708 / 58229
709 / 58229
710 / 58229
711 / 58229
712 / 58229
713 / 58229
714 / 58229
715 / 58229
716 / 58229
717 / 58229
718 / 58229
719 / 58229
720 / 58229
721 / 58229
722 / 58229
723 / 58229
724 / 58229
725 / 58229
726 / 58229
727 / 58229
728 / 58229
729 / 58229
730 / 58229
731 / 58229
732 / 58229
733 / 58229
734 / 58229
735 / 58229
736 / 58229
737 / 58229
738 / 58229
739 / 58229
740 / 58229
741 / 58229
742 / 58229
743 / 58229
744 / 58229
745 / 58229
746 / 58229
747 / 58229
748 / 58229
749 / 58229
750 / 58229
751 / 58229
752 / 58229
753 / 58229
754 / 58229
755 / 58229
756 / 58229
757 / 58229
758 / 58229
759 / 58229
760 / 58229
761 / 58229
762 / 58229
763 / 58229
764 / 58229
765 / 58229
766 / 58229
767 / 58229
768 / 58229
769 / 58229
770 / 58229
771 / 58229
772 / 58229
773 / 58229
774 / 58229
775 / 58229
776 / 58229
777 / 58229
778 / 58229
779 / 58229
780 / 58229
781 / 58229
782 / 58229
783 / 58229
784 / 58229
785 / 58229
786 / 58229
787 

1372 / 58229
1373 / 58229
1374 / 58229
1375 / 58229
1376 / 58229
1377 / 58229
1378 / 58229
1379 / 58229
1380 / 58229
1381 / 58229
1382 / 58229
1383 / 58229
1384 / 58229
1385 / 58229
1386 / 58229
1387 / 58229
1388 / 58229
1389 / 58229
1390 / 58229
1391 / 58229
1392 / 58229
1393 / 58229
1394 / 58229
1395 / 58229
1396 / 58229
1397 / 58229
1398 / 58229
1399 / 58229
1400 / 58229
1401 / 58229
1402 / 58229
1403 / 58229
1404 / 58229
1405 / 58229
1406 / 58229
1407 / 58229
1408 / 58229
1409 / 58229
1410 / 58229
1411 / 58229
1412 / 58229
1413 / 58229
1414 / 58229
1415 / 58229
1416 / 58229
1417 / 58229
1418 / 58229
1419 / 58229
1420 / 58229
1421 / 58229
1422 / 58229
1423 / 58229
1424 / 58229
1425 / 58229
1426 / 58229
1427 / 58229
1428 / 58229
1429 / 58229
1430 / 58229
1431 / 58229
1432 / 58229
1433 / 58229
1434 / 58229
1435 / 58229
1436 / 58229
1437 / 58229
1438 / 58229
1439 / 58229
1440 / 58229
1441 / 58229
1442 / 58229
1443 / 58229
1444 / 58229
1445 / 58229
1446 / 58229
1447 / 58229
1448 / 58229

2004 / 58229
2005 / 58229
2006 / 58229
2007 / 58229
2008 / 58229
2009 / 58229
2010 / 58229
2011 / 58229
2012 / 58229
2013 / 58229
2014 / 58229
2015 / 58229
2016 / 58229
2017 / 58229
2018 / 58229
2019 / 58229
2020 / 58229
2021 / 58229
2022 / 58229
2023 / 58229
2024 / 58229
2025 / 58229
2026 / 58229
2027 / 58229
2028 / 58229
2029 / 58229
2030 / 58229
2031 / 58229
2032 / 58229
2033 / 58229
2034 / 58229
2035 / 58229
2036 / 58229
2037 / 58229
2038 / 58229
2039 / 58229
2040 / 58229
2041 / 58229
2042 / 58229
2043 / 58229
2044 / 58229
2045 / 58229
2046 / 58229
2047 / 58229
2048 / 58229
2049 / 58229
2050 / 58229
2051 / 58229
2052 / 58229
2053 / 58229
2054 / 58229
2055 / 58229
2056 / 58229
2057 / 58229
2058 / 58229
2059 / 58229
2060 / 58229
2061 / 58229
2062 / 58229
2063 / 58229
2064 / 58229
2065 / 58229
2066 / 58229
2067 / 58229
2068 / 58229
2069 / 58229
2070 / 58229
2071 / 58229
2072 / 58229
2073 / 58229
2074 / 58229
2075 / 58229
2076 / 58229
2077 / 58229
2078 / 58229
2079 / 58229
2080 / 58229

2637 / 58229
2638 / 58229
2639 / 58229
2640 / 58229
2641 / 58229
2642 / 58229
2643 / 58229
2644 / 58229
2645 / 58229
2646 / 58229
2647 / 58229
2648 / 58229
2649 / 58229
2650 / 58229
2651 / 58229
2652 / 58229
2653 / 58229
2654 / 58229
2655 / 58229
2656 / 58229
2657 / 58229
2658 / 58229
2659 / 58229
2660 / 58229
2661 / 58229
2662 / 58229
2663 / 58229
2664 / 58229
2665 / 58229
2666 / 58229
2667 / 58229
2668 / 58229
2669 / 58229
2670 / 58229
2671 / 58229
2672 / 58229
2673 / 58229
2674 / 58229
2675 / 58229
2676 / 58229
2677 / 58229
2678 / 58229
2679 / 58229
2680 / 58229
2681 / 58229
2682 / 58229
2683 / 58229
2684 / 58229
2685 / 58229
2686 / 58229
2687 / 58229
2688 / 58229
2689 / 58229
2690 / 58229
2691 / 58229
2692 / 58229
2693 / 58229
2694 / 58229
2695 / 58229
2696 / 58229
2697 / 58229
2698 / 58229
2699 / 58229
2700 / 58229
2701 / 58229
2702 / 58229
2703 / 58229
2704 / 58229
2705 / 58229
2706 / 58229
2707 / 58229
2708 / 58229
2709 / 58229
2710 / 58229
2711 / 58229
2712 / 58229
2713 / 58229

...

52026 / 58229
52027 / 58229
52028 / 58229
52029 / 58229
52030 / 58229
52031 / 58229
52032 / 58229
52033 / 58229
52034 / 58229
52035 / 58229
52036 / 58229
52037 / 58229
52038 / 58229
52039 / 58229
52040 / 58229
52041 / 58229
52042 / 58229
52043 / 58229
52044 / 58229
52045 / 58229
52046 / 58229
52047 / 58229
52048 / 58229
52049 / 58229
52050 / 58229
52051 / 58229
52052 / 58229
52053 / 58229
52054 / 58229
52055 / 58229
52056 / 58229
52057 / 58229
52058 / 58229
52059 / 58229
52060 / 58229
52061 / 58229
52062 / 58229
52063 / 58229
52064 / 58229
52065 / 58229
52066 / 58229
52067 / 58229
52068 / 58229
52069 / 58229
52070 / 58229
52071 / 58229
52072 / 58229
52073 / 58229
52074 / 58229
52075 / 58229
52076 / 58229
52077 / 58229
52078 / 58229
52079 / 58229
52080 / 58229
52081 / 58229
52082 / 58229
52083 / 58229
52084 / 58229
52085 / 58229
52086 / 58229
52087 / 58229
52088 / 58229
52089 / 58229
52090 / 58229
52091 / 58229
52092 / 58229
52093 / 58229
52094 / 58229
52095 / 58229
52096 / 58229
52097 

52624 / 58229
52625 / 58229
52626 / 58229
52627 / 58229
52628 / 58229
52629 / 58229
52630 / 58229
52631 / 58229
52632 / 58229
52633 / 58229
52634 / 58229
52635 / 58229
52636 / 58229
52637 / 58229
52638 / 58229
52639 / 58229
52640 / 58229
52641 / 58229
52642 / 58229
52643 / 58229
52644 / 58229
52645 / 58229
52646 / 58229
52647 / 58229
52648 / 58229
52649 / 58229
52650 / 58229
52651 / 58229
52652 / 58229
52653 / 58229
52654 / 58229
52655 / 58229
52656 / 58229
52657 / 58229
52658 / 58229
52659 / 58229
52660 / 58229
52661 / 58229
52662 / 58229
52663 / 58229
52664 / 58229
52665 / 58229
52666 / 58229
52667 / 58229
52668 / 58229
52669 / 58229
52670 / 58229
52671 / 58229
52672 / 58229
52673 / 58229
52674 / 58229
52675 / 58229
52676 / 58229
52677 / 58229
52678 / 58229
52679 / 58229
52680 / 58229
52681 / 58229
52682 / 58229
52683 / 58229
52684 / 58229
52685 / 58229
52686 / 58229
52687 / 58229
52688 / 58229
52689 / 58229
52690 / 58229
52691 / 58229
52692 / 58229
52693 / 58229
52694 / 58229
52695 

53219 / 58229
53220 / 58229
53221 / 58229
53222 / 58229
53223 / 58229
53224 / 58229
53225 / 58229
53226 / 58229
53227 / 58229
53228 / 58229
53229 / 58229
53230 / 58229
53231 / 58229
53232 / 58229
53233 / 58229
53234 / 58229
53235 / 58229
53236 / 58229
53237 / 58229
53238 / 58229
53239 / 58229
53240 / 58229
53241 / 58229
53242 / 58229
53243 / 58229
53244 / 58229
53245 / 58229
53246 / 58229
53247 / 58229
53248 / 58229
53249 / 58229
53250 / 58229
53251 / 58229
53252 / 58229
53253 / 58229
53254 / 58229
53255 / 58229
53256 / 58229
53257 / 58229
53258 / 58229
53259 / 58229
53260 / 58229
53261 / 58229
53262 / 58229
53263 / 58229
53264 / 58229
53265 / 58229
53266 / 58229
53267 / 58229
53268 / 58229
53269 / 58229
53270 / 58229
53271 / 58229
53272 / 58229
53273 / 58229
53274 / 58229
53275 / 58229
53276 / 58229
53277 / 58229
53278 / 58229
53279 / 58229
53280 / 58229
53281 / 58229
53282 / 58229
53283 / 58229
53284 / 58229
53285 / 58229
53286 / 58229
53287 / 58229
53288 / 58229
53289 / 58229
53290 

53822 / 58229
53823 / 58229
53824 / 58229
53825 / 58229
53826 / 58229
53827 / 58229
53828 / 58229
53829 / 58229
53830 / 58229
53831 / 58229
53832 / 58229
53833 / 58229
53834 / 58229
53835 / 58229
53836 / 58229
53837 / 58229
53838 / 58229
53839 / 58229
53840 / 58229
53841 / 58229
53842 / 58229
53843 / 58229
53844 / 58229
53845 / 58229
53846 / 58229
53847 / 58229
53848 / 58229
53849 / 58229
53850 / 58229
53851 / 58229
53852 / 58229
53853 / 58229
53854 / 58229
53855 / 58229
53856 / 58229
53857 / 58229
53858 / 58229
53859 / 58229
53860 / 58229
53861 / 58229
53862 / 58229
53863 / 58229
53864 / 58229
53865 / 58229
53866 / 58229
53867 / 58229
53868 / 58229
53869 / 58229
53870 / 58229
53871 / 58229
53872 / 58229
53873 / 58229
53874 / 58229
53875 / 58229
53876 / 58229
53877 / 58229
53878 / 58229
53879 / 58229
53880 / 58229
53881 / 58229
53882 / 58229
53883 / 58229
53884 / 58229
53885 / 58229
53886 / 58229
53887 / 58229
53888 / 58229
53889 / 58229
53890 / 58229
53891 / 58229
53892 / 58229
53893 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [257]:
train_dataset.drop(train_dataset.index[index_for_removal], inplace=True)
print(train_dataset.shape)
train_dataset.head()

(27618, 25)


,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
23511,0.605146,0.277083,0.333470,0.648512,0.320145,0.323761,0.172499,0.125225,0.236151,0.223894,...,-0.074089,-0.127328,-0.129829,-0.108098,-0.098851,0.009330,0.014946,0.028910,0.008939,0.005300
3986,0.957899,0.535175,0.452014,0.379482,0.473959,0.481866,0.325451,0.304671,0.236151,0.223894,...,-0.083324,-0.038124,-0.032183,-0.118196,-0.066099,0.018335,0.013545,0.008846,0.005404,0.010136
13927,0.391024,0.335950,0.334008,0.342975,0.286672,0.170522,0.390207,0.236212,0.236151,0.223894,...,-0.150164,-0.143323,-0.191539,-0.132769,-0.150759,0.016779,0.014639,0.027927,0.032017,0.010331
26305,0.577592,0.301341,0.286958,0.319033,0.393285,0.392008,0.260916,0.255716,0.236151,0.223894,...,-0.075841,-0.014383,-0.012429,-0.074499,-0.045789,0.004704,0.006357,0.006290,0.010915,0.004229
27443,0.554276,0.286676,0.271690,0.232989,0.389926,0.378093,0.265508,0.252122,0.236151,0.223894,...,-0.018438,-0.038318,-0.080611,-0.011859,-0.033967,0.002183,0.002089,0.003746,0.000961,0.002483


In [258]:
train_emotions.drop(train_emotions.index[index_for_removal], inplace=True)
print(train_emotions.shape)
train_emotions.head()

(27618, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
2,0.600116,0.402414,0.101721
5,0.599353,0.401356,0.096966
6,0.099969,-0.754206,-0.255016
8,0.099685,-0.746098,-0.237900
12,-0.099119,-0.594414,-0.149684


In [259]:
train_emotions_OG.drop(train_emotions_OG.index[index_for_removal], inplace=True)
print(train_emotions_OG.shape)
train_emotions_OG.head()

(27618, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
23511,0.6,0.40,0.10
3986,0.6,0.40,0.10
13927,0.1,-0.75,-0.25
26305,0.1,-0.75,-0.25
27443,-0.1,-0.60,-0.15


In [260]:
# Remove LMA features whose predictions are too different from the real one
generated_decoded_test = autoencoder.decoder.predict(generated_test)

index_for_removal = []


for ind in range(0,generated_decoded_test.shape[0]):
    print(str(ind) + " / " + str(generated_decoded_test.shape[0]))
    predicted = np.asarray([generated_decoded_test[ind]])
    true = np.asarray([test_emotions.iloc[ind]])

    predicted = scaler.transform(predicted)

    predicted = (
        model_p.predict(predicted),
        model_a.predict(predicted),
        model_d.predict(predicted)
    )
    
    predicted = np.asarray(predicted).transpose()
    
    #print(predicted)
    #print(true)
    
    mae_errors = mean_absolute_error(true, predicted, multioutput='raw_values')
    #print(mae_errors)
    
    if(mae_errors[0] > 0.25 or mae_errors[1] > 0.25 or mae_errors[2] > 0.25):
        index_for_removal.append(ind)
        continue
    
    mae_error = mean_absolute_error(true, predicted)
    #print(mae_error)
    
    if(mae_error > 0.15):
        index_for_removal.append(ind)
        continue
    
    #print()

print(len(index_for_removal))


0 / 14564
1 / 14564
2 / 14564
3 / 14564
4 / 14564
5 / 14564
6 / 14564
7 / 14564
8 / 14564
9 / 14564
10 / 14564
11 / 14564
12 / 14564
13 / 14564
14 / 14564
15 / 14564
16 / 14564
17 / 14564
18 / 14564
19 / 14564
20 / 14564
21 / 14564
22 / 14564
23 / 14564
24 / 14564
25 / 14564
26 / 14564
27 / 14564
28 / 14564
29 / 14564
30 / 14564
31 / 14564
32 / 14564
33 / 14564
34 / 14564
35 / 14564
36 / 14564
37 / 14564
38 / 14564
39 / 14564
40 / 14564
41 / 14564
42 / 14564
43 / 14564
44 / 14564
45 / 14564
46 / 14564
47 / 14564
48 / 14564
49 / 14564
50 / 14564
51 / 14564
52 / 14564
53 / 14564
54 / 14564
55 / 14564
56 / 14564
57 / 14564
58 / 14564
59 / 14564
60 / 14564
61 / 14564
62 / 14564
63 / 14564
64 / 14564
65 / 14564
66 / 14564
67 / 14564
68 / 14564
69 / 14564
70 / 14564
71 / 14564
72 / 14564
73 / 14564
74 / 14564
75 / 14564
76 / 14564
77 / 14564
78 / 14564
79 / 14564
80 / 14564
81 / 14564
82 / 14564
83 / 14564
84 / 14564
85 / 14564
86 / 14564
87 / 14564
88 / 14564
89 / 14564
90 / 14564
91 / 1456

709 / 14564
710 / 14564
711 / 14564
712 / 14564
713 / 14564
714 / 14564
715 / 14564
716 / 14564
717 / 14564
718 / 14564
719 / 14564
720 / 14564
721 / 14564
722 / 14564
723 / 14564
724 / 14564
725 / 14564
726 / 14564
727 / 14564
728 / 14564
729 / 14564
730 / 14564
731 / 14564
732 / 14564
733 / 14564
734 / 14564
735 / 14564
736 / 14564
737 / 14564
738 / 14564
739 / 14564
740 / 14564
741 / 14564
742 / 14564
743 / 14564
744 / 14564
745 / 14564
746 / 14564
747 / 14564
748 / 14564
749 / 14564
750 / 14564
751 / 14564
752 / 14564
753 / 14564
754 / 14564
755 / 14564
756 / 14564
757 / 14564
758 / 14564
759 / 14564
760 / 14564
761 / 14564
762 / 14564
763 / 14564
764 / 14564
765 / 14564
766 / 14564
767 / 14564
768 / 14564
769 / 14564
770 / 14564
771 / 14564
772 / 14564
773 / 14564
774 / 14564
775 / 14564
776 / 14564
777 / 14564
778 / 14564
779 / 14564
780 / 14564
781 / 14564
782 / 14564
783 / 14564
784 / 14564
785 / 14564
786 / 14564
787 / 14564
788 / 14564
789 / 14564
790 / 14564
791 / 14564
792 

1363 / 14564
1364 / 14564
1365 / 14564
1366 / 14564
1367 / 14564
1368 / 14564
1369 / 14564
1370 / 14564
1371 / 14564
1372 / 14564
1373 / 14564
1374 / 14564
1375 / 14564
1376 / 14564
1377 / 14564
1378 / 14564
1379 / 14564
1380 / 14564
1381 / 14564
1382 / 14564
1383 / 14564
1384 / 14564
1385 / 14564
1386 / 14564
1387 / 14564
1388 / 14564
1389 / 14564
1390 / 14564
1391 / 14564
1392 / 14564
1393 / 14564
1394 / 14564
1395 / 14564
1396 / 14564
1397 / 14564
1398 / 14564
1399 / 14564
1400 / 14564
1401 / 14564
1402 / 14564
1403 / 14564
1404 / 14564
1405 / 14564
1406 / 14564
1407 / 14564
1408 / 14564
1409 / 14564
1410 / 14564
1411 / 14564
1412 / 14564
1413 / 14564
1414 / 14564
1415 / 14564
1416 / 14564
1417 / 14564
1418 / 14564
1419 / 14564
1420 / 14564
1421 / 14564
1422 / 14564
1423 / 14564
1424 / 14564
1425 / 14564
1426 / 14564
1427 / 14564
1428 / 14564
1429 / 14564
1430 / 14564
1431 / 14564
1432 / 14564
1433 / 14564
1434 / 14564
1435 / 14564
1436 / 14564
1437 / 14564
1438 / 14564
1439 / 14564

1997 / 14564
1998 / 14564
1999 / 14564
2000 / 14564
2001 / 14564
2002 / 14564
2003 / 14564
2004 / 14564
2005 / 14564
2006 / 14564
2007 / 14564
2008 / 14564
2009 / 14564
2010 / 14564
2011 / 14564
2012 / 14564
2013 / 14564
2014 / 14564
2015 / 14564
2016 / 14564
2017 / 14564
2018 / 14564
2019 / 14564
2020 / 14564
2021 / 14564
2022 / 14564
2023 / 14564
2024 / 14564
2025 / 14564
2026 / 14564
2027 / 14564
2028 / 14564
2029 / 14564
2030 / 14564
2031 / 14564
2032 / 14564
2033 / 14564
2034 / 14564
2035 / 14564
2036 / 14564
2037 / 14564
2038 / 14564
2039 / 14564
2040 / 14564
2041 / 14564
2042 / 14564
2043 / 14564
2044 / 14564
2045 / 14564
2046 / 14564
2047 / 14564
2048 / 14564
2049 / 14564
2050 / 14564
2051 / 14564
2052 / 14564
2053 / 14564
2054 / 14564
2055 / 14564
2056 / 14564
2057 / 14564
2058 / 14564
2059 / 14564
2060 / 14564
2061 / 14564
2062 / 14564
2063 / 14564
2064 / 14564
2065 / 14564
2066 / 14564
2067 / 14564
2068 / 14564
2069 / 14564
2070 / 14564
2071 / 14564
2072 / 14564
2073 / 14564

2629 / 14564
2630 / 14564
2631 / 14564
2632 / 14564
2633 / 14564
2634 / 14564
2635 / 14564
2636 / 14564
2637 / 14564
2638 / 14564
2639 / 14564
2640 / 14564
2641 / 14564
2642 / 14564
2643 / 14564
2644 / 14564
2645 / 14564
2646 / 14564
2647 / 14564
2648 / 14564
2649 / 14564
2650 / 14564
2651 / 14564
2652 / 14564
2653 / 14564
2654 / 14564
2655 / 14564
2656 / 14564
2657 / 14564
2658 / 14564
2659 / 14564
2660 / 14564
2661 / 14564
2662 / 14564
2663 / 14564
2664 / 14564
2665 / 14564
2666 / 14564
2667 / 14564
2668 / 14564
2669 / 14564
2670 / 14564
2671 / 14564
2672 / 14564
2673 / 14564
2674 / 14564
2675 / 14564
2676 / 14564
2677 / 14564
2678 / 14564
2679 / 14564
2680 / 14564
2681 / 14564
2682 / 14564
2683 / 14564
2684 / 14564
2685 / 14564
2686 / 14564
2687 / 14564
2688 / 14564
2689 / 14564
2690 / 14564
2691 / 14564
2692 / 14564
2693 / 14564
2694 / 14564
2695 / 14564
2696 / 14564
2697 / 14564
2698 / 14564
2699 / 14564
2700 / 14564
2701 / 14564
2702 / 14564
2703 / 14564
2704 / 14564
2705 / 14564

...

12053 / 14564
12054 / 14564
12055 / 14564
12056 / 14564
12057 / 14564
12058 / 14564
12059 / 14564
12060 / 14564
12061 / 14564
12062 / 14564
12063 / 14564
12064 / 14564
12065 / 14564
12066 / 14564
12067 / 14564
12068 / 14564
12069 / 14564
12070 / 14564
12071 / 14564
12072 / 14564
12073 / 14564
12074 / 14564
12075 / 14564
12076 / 14564
12077 / 14564
12078 / 14564
12079 / 14564
12080 / 14564
12081 / 14564
12082 / 14564
12083 / 14564
12084 / 14564
12085 / 14564
12086 / 14564
12087 / 14564
12088 / 14564
12089 / 14564
12090 / 14564
12091 / 14564
12092 / 14564
12093 / 14564
12094 / 14564
12095 / 14564
12096 / 14564
12097 / 14564
12098 / 14564
12099 / 14564
12100 / 14564
12101 / 14564
12102 / 14564
12103 / 14564
12104 / 14564
12105 / 14564
12106 / 14564
12107 / 14564
12108 / 14564
12109 / 14564
12110 / 14564
12111 / 14564
12112 / 14564
12113 / 14564
12114 / 14564
12115 / 14564
12116 / 14564
12117 / 14564
12118 / 14564
12119 / 14564
12120 / 14564
12121 / 14564
12122 / 14564
12123 / 14564
12124 

12647 / 14564
12648 / 14564
12649 / 14564
12650 / 14564
12651 / 14564
12652 / 14564
12653 / 14564
12654 / 14564
12655 / 14564
12656 / 14564
12657 / 14564
12658 / 14564
12659 / 14564
12660 / 14564
12661 / 14564
12662 / 14564
12663 / 14564
12664 / 14564
12665 / 14564
12666 / 14564
12667 / 14564
12668 / 14564
12669 / 14564
12670 / 14564
12671 / 14564
12672 / 14564
12673 / 14564
12674 / 14564
12675 / 14564
12676 / 14564
12677 / 14564
12678 / 14564
12679 / 14564
12680 / 14564
12681 / 14564
12682 / 14564
12683 / 14564
12684 / 14564
12685 / 14564
12686 / 14564
12687 / 14564
12688 / 14564
12689 / 14564
12690 / 14564
12691 / 14564
12692 / 14564
12693 / 14564
12694 / 14564
12695 / 14564
12696 / 14564
12697 / 14564
12698 / 14564
12699 / 14564
12700 / 14564
12701 / 14564
12702 / 14564
12703 / 14564
12704 / 14564
12705 / 14564
12706 / 14564
12707 / 14564
12708 / 14564
12709 / 14564
12710 / 14564
12711 / 14564
12712 / 14564
12713 / 14564
12714 / 14564
12715 / 14564
12716 / 14564
12717 / 14564
12718 

13242 / 14564
13243 / 14564
13244 / 14564
13245 / 14564
13246 / 14564
13247 / 14564
13248 / 14564
13249 / 14564
13250 / 14564
13251 / 14564
13252 / 14564
13253 / 14564
13254 / 14564
13255 / 14564
13256 / 14564
13257 / 14564
13258 / 14564
13259 / 14564
13260 / 14564
13261 / 14564
13262 / 14564
13263 / 14564
13264 / 14564
13265 / 14564
13266 / 14564
13267 / 14564
13268 / 14564
13269 / 14564
13270 / 14564
13271 / 14564
13272 / 14564
13273 / 14564
13274 / 14564
13275 / 14564
13276 / 14564
13277 / 14564
13278 / 14564
13279 / 14564
13280 / 14564
13281 / 14564
13282 / 14564
13283 / 14564
13284 / 14564
13285 / 14564
13286 / 14564
13287 / 14564
13288 / 14564
13289 / 14564
13290 / 14564
13291 / 14564
13292 / 14564
13293 / 14564
13294 / 14564
13295 / 14564
13296 / 14564
13297 / 14564
13298 / 14564
13299 / 14564
13300 / 14564
13301 / 14564
13302 / 14564
13303 / 14564
13304 / 14564
13305 / 14564
13306 / 14564
13307 / 14564
13308 / 14564
13309 / 14564
13310 / 14564
13311 / 14564
13312 / 14564
13313 

13838 / 14564
13839 / 14564
13840 / 14564
13841 / 14564
13842 / 14564
13843 / 14564
13844 / 14564
13845 / 14564
13846 / 14564
13847 / 14564
13848 / 14564
13849 / 14564
13850 / 14564
13851 / 14564
13852 / 14564
13853 / 14564
13854 / 14564
13855 / 14564
13856 / 14564
13857 / 14564
13858 / 14564
13859 / 14564
13860 / 14564
13861 / 14564
13862 / 14564
13863 / 14564
13864 / 14564
13865 / 14564
13866 / 14564
13867 / 14564
13868 / 14564
13869 / 14564
13870 / 14564
13871 / 14564
13872 / 14564
13873 / 14564
13874 / 14564
13875 / 14564
13876 / 14564
13877 / 14564
13878 / 14564
13879 / 14564
13880 / 14564
13881 / 14564
13882 / 14564
13883 / 14564
13884 / 14564
13885 / 14564
13886 / 14564
13887 / 14564
13888 / 14564
13889 / 14564
13890 / 14564
13891 / 14564
13892 / 14564
13893 / 14564
13894 / 14564
13895 / 14564
13896 / 14564
13897 / 14564
13898 / 14564
13899 / 14564
13900 / 14564
13901 / 14564
13902 / 14564
13903 / 14564
13904 / 14564
13905 / 14564
13906 / 14564
13907 / 14564
13908 / 14564
13909 

14435 / 14564
14436 / 14564
14437 / 14564
14438 / 14564
14439 / 14564
14440 / 14564
14441 / 14564
14442 / 14564
14443 / 14564
14444 / 14564
14445 / 14564
14446 / 14564
14447 / 14564
14448 / 14564
14449 / 14564
14450 / 14564
14451 / 14564
14452 / 14564
14453 / 14564
14454 / 14564
14455 / 14564
14456 / 14564
14457 / 14564
14458 / 14564
14459 / 14564
14460 / 14564
14461 / 14564
14462 / 14564
14463 / 14564
14464 / 14564
14465 / 14564
14466 / 14564
14467 / 14564
14468 / 14564
14469 / 14564
14470 / 14564
14471 / 14564
14472 / 14564
14473 / 14564
14474 / 14564
14475 / 14564
14476 / 14564
14477 / 14564
14478 / 14564
14479 / 14564
14480 / 14564
14481 / 14564
14482 / 14564
14483 / 14564
14484 / 14564
14485 / 14564
14486 / 14564
14487 / 14564
14488 / 14564
14489 / 14564
14490 / 14564
14491 / 14564
14492 / 14564
14493 / 14564
14494 / 14564
14495 / 14564
14496 / 14564
14497 / 14564
14498 / 14564
14499 / 14564
14500 / 14564
14501 / 14564
14502 / 14564
14503 / 14564
14504 / 14564
14505 / 14564
14506 

In [261]:
test_dataset.drop(test_dataset.index[index_for_removal], inplace=True)
print(test_dataset.shape)
test_dataset.head()

(6956, 25)


,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
56226,0.574909,0.307601,0.272455,0.410620,0.380857,0.378913,0.271549,0.269376,0.236151,0.223894,...,-0.051414,-0.036004,-0.025033,-0.065891,-0.045062,0.004328,0.009201,0.007693,0.015086,0.006038
65859,0.740076,0.409941,0.340451,0.295422,0.413895,0.422891,0.320570,0.291677,0.236151,0.223894,...,-0.178694,-0.120949,-0.095157,-0.196285,-0.143627,0.036501,0.035673,0.029382,0.005784,0.025115
19700,0.599736,0.460908,0.340772,0.867680,0.347605,0.292026,0.094260,0.344792,0.236151,0.223894,...,-0.188515,-0.190831,-0.279597,-0.174220,-0.195540,0.026340,0.019924,0.007631,0.028016,0.017682
66623,0.583907,0.287681,0.296032,0.372244,0.395994,0.384544,0.258207,0.266992,0.236151,0.223894,...,-0.047052,-0.055775,-0.037993,-0.034307,-0.050051,0.013003,0.007396,0.014827,0.010149,0.006818
25344,0.631201,0.499927,0.269318,0.461293,0.383338,0.258622,0.137204,0.258157,0.236151,0.223894,...,-0.247892,-0.255305,-0.248253,-0.154293,-0.232245,0.018642,0.020121,0.065263,0.030015,0.025581


In [262]:
test_emotions.drop(test_emotions.index[index_for_removal], inplace=True)
print(test_emotions.shape)
test_emotions.head()

(6956, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
1,-0.100236,-0.596791,-0.154467
2,0.299907,0.398420,0.595462
3,0.100203,0.604365,0.402592
7,-0.099456,-0.596691,-0.146766
8,0.100378,0.607368,0.401693


In [263]:
test_emotions_OG.drop(test_emotions_OG.index[index_for_removal], inplace=True)
print(test_emotions_OG.shape)
test_emotions_OG.head()

(6956, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
56226,-0.1,-0.6,-0.15
65859,0.3,0.4,0.60
19700,0.1,0.6,0.40
66623,-0.1,-0.6,-0.15
25344,0.1,0.6,0.40


In [264]:
generated_train = autoencoder.encoder.predict(train_dataset)
generated_test = autoencoder.encoder.predict(test_dataset)

generated_train_x = train_emotions.copy()
generated_test_x = test_emotions.copy()
#generated_train_x = train_emotions_OG.copy()
#generated_test_x = test_emotions_OG.copy()

## XGB

In [265]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

import xgboost as xgb

xgb.set_config(verbosity=0)

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import normalize

from sklearn.utils import shuffle
import math
import joblib
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import time

In [266]:
generated_test_1 = generated_test[:,0]
generated_train_1 = generated_train[:,0]

generated_test_2 = generated_test[:,1]
generated_train_2 = generated_train[:,1]

generated_test_3 = generated_test[:,2]
generated_train_3 = generated_train[:,2]

generated_test_4 = generated_test[:,3]
generated_train_4 = generated_train[:,3]

generated_test_5 = generated_test[:,4]
generated_train_5 = generated_train[:,4]

In [267]:
xgbr = xgb.XGBRegressor(verbosity=1) 

params = {
        'eta': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        'gamma': [0, 0.001, 0.01],
        'subsample': [0.75, 1],
        'colsample_bytree': [0.75, 1],
        'lambda': [1, 1.25],
        'alpha': [0.0, 0.25]
        }

#{'colsample_bytree': 1, 'gamma': 0, 'max_depth': 6, 'min_child_weight': 11, 'subsample': 0.75, 
#lambda': 1.25, 'eta': 0.01, 'alpha': 0.0}
model_1 = xgb.XGBRegressor(
                    n_estimators=5000, learning_rate=0.01, max_depth=6, min_child_weight=11, 
                    reg_alpha=0.0, reg_lambda=1.25, gamma=0.0,
                    subsample=0.75, colsample_bytree=1.0, objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id=0
                )


model_2 = xgb.XGBRegressor(
                    n_estimators=5000, learning_rate=0.01, max_depth=6, min_child_weight=11, 
                    reg_alpha=0.0, reg_lambda=1.25, gamma=0.0,
                    subsample=0.75, colsample_bytree=1.0, objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id=0
                )

    
model_3 = xgb.XGBRegressor(
                    n_estimators=5000, learning_rate=0.01, max_depth=6, min_child_weight=11, 
                    reg_alpha=0.0, reg_lambda=1.25, gamma=0.0,
                    subsample=0.75, colsample_bytree=1.0, objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id=0
                )

model_4 = xgb.XGBRegressor(
                    n_estimators=5000, learning_rate=0.01, max_depth=6, min_child_weight=11, 
                    reg_alpha=0.0, reg_lambda=1.25, gamma=0.0,
                    subsample=0.75, colsample_bytree=1.0, objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id=0
                )

model_5 = xgb.XGBRegressor(
                    n_estimators=5000, learning_rate=0.01, max_depth=6, min_child_weight=11, 
                    reg_alpha=0.0, reg_lambda=1.25, gamma=0.0,
                    subsample=0.75, colsample_bytree=1.0, objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id=0
                )

In [268]:
model_1.fit(generated_train_x, generated_train_1)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, enable_categorical=False,
             gamma=0.0, gpu_id=0, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=11, missing=nan,
             monotone_constraints='()', n_estimators=5000, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.0, reg_lambda=1.25, scale_pos_weight=1, subsample=0.75,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [269]:
model_2.fit(generated_train_x, generated_train_2)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, enable_categorical=False,
             gamma=0.0, gpu_id=0, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=11, missing=nan,
             monotone_constraints='()', n_estimators=5000, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.0, reg_lambda=1.25, scale_pos_weight=1, subsample=0.75,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [270]:
model_3.fit(generated_train_x, generated_train_3)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, enable_categorical=False,
             gamma=0.0, gpu_id=0, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=11, missing=nan,
             monotone_constraints='()', n_estimators=5000, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.0, reg_lambda=1.25, scale_pos_weight=1, subsample=0.75,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [271]:
model_4.fit(generated_train_x, generated_train_4)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, enable_categorical=False,
             gamma=0.0, gpu_id=0, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=11, missing=nan,
             monotone_constraints='()', n_estimators=5000, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.0, reg_lambda=1.25, scale_pos_weight=1, subsample=0.75,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [272]:
model_5.fit(generated_train_x, generated_train_5)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, enable_categorical=False,
             gamma=0.0, gpu_id=0, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=11, missing=nan,
             monotone_constraints='()', n_estimators=5000, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.0, reg_lambda=1.25, scale_pos_weight=1, subsample=0.75,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [273]:
pred_y_1 = model_1.predict(generated_test_x)
mse = mean_squared_error(generated_test_1, pred_y_1)
mae = mean_absolute_error(generated_test_1, pred_y_1)
print("Latent Feature 1")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)


pred_y_2 = model_2.predict(generated_test_x)
mse = mean_squared_error(generated_test_2, pred_y_2)
mae = mean_absolute_error(generated_test_2, pred_y_2)
print("\nLatent Feature 2")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)


pred_y_3 = model_3.predict(generated_test_x)
mse = mean_squared_error(generated_test_3, pred_y_3)
mae = mean_absolute_error(generated_test_3, pred_y_3)
print("\nLatent Feature 3")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

pred_y_4 = model_4.predict(generated_test_x)
mse = mean_squared_error(generated_test_4, pred_y_4)
mae = mean_absolute_error(generated_test_4, pred_y_4)
print("\nLatent Feature 4")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

pred_y_5 = model_5.predict(generated_test_x)
mse = mean_squared_error(generated_test_5, pred_y_5)
mae = mean_absolute_error(generated_test_5, pred_y_5)
print("\nLatent Feature 5")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

Latent Feature 1
MSE: 0.28
MAE: 0.45

Latent Feature 2
MSE: 0.26
MAE: 0.41

Latent Feature 3
MSE: 0.27
MAE: 0.35

Latent Feature 4
MSE: 0.12
MAE: 0.27

Latent Feature 5
MSE: 0.10
MAE: 0.25


In [274]:
index = 0

row=np.asarray([generated_test_x.iloc[index]])
#row = np.asarray([[0.0, 0.0, 0.0]])

y_1 = model_1.predict(row)
y_2 = model_2.predict(row)
y_3 = model_3.predict(row)
y_4 = model_4.predict(row)
y_5 = model_5.predict(row)

In [275]:
sample = np.asarray([[y_1[0], y_2[0], y_3[0], y_4[0], y_5[0]]])
og_coords = generated_test_x.iloc[index]
print(og_coords)
print(sample)

EMOTION_P   -0.100236
EMOTION_A   -0.596791
EMOTION_D   -0.154467
Name: 1, dtype: float32
[[-0.08600271 -0.17629492  0.18828182 -0.12695153 -0.16884805]]


In [276]:
regen = autoencoder.decoder.predict(sample)
print(regen)

[[ 0.6030632   0.29005915  0.40464813  0.2598688   0.35308284  0.39890596
   0.11556718  0.17233185  0.2363371   0.22392768  0.22375223  0.08613995
   0.04683933  0.11053158  0.09872969 -0.10250087 -0.01445765  0.00841166
  -0.08787949 -0.04993176  0.01260324  0.01377389  0.01100974  0.00671326
   0.0072244 ]]


In [277]:
scaled_regen = scaler.transform(regen)

generated_coordinates = (
    model_p.predict(scaled_regen),
    model_a.predict(scaled_regen),
    model_d.predict(scaled_regen)
)


print('Real: %s' % np.asarray(og_coords))
print('Generated: %s' % np.asarray(generated_coordinates).transpose()[0])

Real: [-0.10023569 -0.5967911  -0.15446682]
Generated: [ 0.22553831 -0.4030813  -0.11492181]


### Test

In [278]:
# 1st - Generate each lma feature for each X
generated_features = []

pred_feature = model_1.predict(generated_test_x)
generated_features.append(pred_feature)

pred_feature = model_2.predict(generated_test_x)
generated_features.append(pred_feature)

pred_feature = model_3.predict(generated_test_x)
generated_features.append(pred_feature)

pred_feature = model_4.predict(generated_test_x)
generated_features.append(pred_feature)

pred_feature = model_5.predict(generated_test_x)
generated_features.append(pred_feature)

generated_features = np.asarray(generated_features).transpose()

print(generated_features.shape)

(6956, 5)


In [279]:
# 2nd - Use Decoder to get LMA features
generated_lma = autoencoder.decoder.predict(generated_features)
print(generated_lma.shape)

scaled_generated_lma = scaler.transform(generated_lma)

(6956, 25)


In [280]:
# 3rd - Get Predicted Emotions
gen_emotions_p = model_p.predict(scaled_generated_lma)
gen_emotions_a = model_a.predict(scaled_generated_lma)
gen_emotions_d = model_d.predict(scaled_generated_lma)

rows = []
for i in range(len(scaled_generated_lma)):
    rows.append([gen_emotions_p[i], gen_emotions_a[i], gen_emotions_d[i]])

gen_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

gen_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.225538,-0.403081,-0.114922
1,0.384092,0.230443,0.090246
2,0.116496,-0.045417,0.152171
3,0.188253,-0.458716,-0.177006
4,0.027694,-0.052973,-0.042343


In [281]:
# 4th - Get MAE
import random

mae_errors = mean_absolute_error(test_emotions, gen_emotions, multioutput='raw_values')
mse_errors = mean_squared_error(test_emotions, gen_emotions, multioutput='raw_values')

features = ["PLEASURE", "AROUSAL", "DOMINANCE"
         ]

print("Overall MAE: " + str(mean_absolute_error(test_emotions, gen_emotions)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print()
    
for i in range(30):
    row = random.randint(0, len(test_emotions))

    print("Real: " + str([test_emotions_OG.iloc[row,0], test_emotions_OG.iloc[row,1], test_emotions_OG.iloc[row,2]]))
    print("Predicted: " + str([test_emotions.iloc[row,0], test_emotions.iloc[row,1], test_emotions.iloc[row,2]]))
    print("Generated: " + str([gen_emotions.iloc[row,0], gen_emotions.iloc[row,1], gen_emotions.iloc[row,2]]))
    print()

Overall MAE: 0.19415444

==PLEASURE==
MSE: 0.05644
MAE: 0.19477

==AROUSAL==
MSE: 0.09223
MAE: 0.24363

==DOMINANCE==
MSE: 0.04757
MAE: 0.14406

Real: [0.05, -0.05, 0.0]
Predicted: [0.05023361, -0.051032733, -0.00078506913]
Generated: [0.08787959, 0.11023198, 0.1204706]

Real: [-0.1, -0.6, -0.15]
Predicted: [-0.10088522, -0.6005038, -0.15238056]
Generated: [0.09326621, -0.6340719, -0.2626434]

Real: [-0.1, -0.6, -0.15]
Predicted: [-0.09647032, -0.5696409, -0.12591825]
Generated: [0.28523678, -0.17376012, -0.068695255]

Real: [0.3, 0.4, 0.6]
Predicted: [0.29420802, 0.3772197, 0.5651554]
Generated: [0.3091644, 0.22153348, 0.14086235]

Real: [-0.1, -0.6, -0.15]
Predicted: [-0.10065629, -0.598679, -0.14637206]
Generated: [0.025150746, -0.44481364, -0.2810939]

Real: [0.1, -0.75, -0.25]
Predicted: [0.099791266, -0.7486841, -0.24650903]
Generated: [0.15169112, -0.65094775, -0.18863937]

Real: [-0.1, -0.6, -0.15]
Predicted: [-0.09996631, -0.5946559, -0.14616038]
Generated: [0.21802202, -0.320

In [ ]:
# Error was 0.24 on AE with 4096 epochs

## Save Models

In [283]:
model_1.save_model("models/bandai/5frame/ae/bandai_pad2l1_model.json")
model_2.save_model("models/bandai/5frame/ae/bandai_pad2l2_model.json")
model_3.save_model("models/bandai/5frame/ae/bandai_pad2l3_model.json")
model_4.save_model("models/bandai/5frame/ae/bandai_pad2l4_model.json")
model_5.save_model("models/bandai/5frame/ae/bandai_pad2l5_model.json")

## RandomGridSearchCV

In [172]:
# A parameter grid for XGBoost
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
params = {
        'eta': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        #'gamma': [0, 0.001],
        'subsample': [0.75, 1],
        'colsample_bytree': [0.75, 1],
        'lambda': [1, 1.25],
        'alpha': [0.0, 0.25]
        }

n_iter = 150

In [173]:
model_1 = xgb.XGBRegressor(
                    n_estimators=2000,
                    objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id=0
                )

model_2 = xgb.XGBRegressor(
                    n_estimators=2000,
                    objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id=0
                )

model_3 = xgb.XGBRegressor(
                    n_estimators=2000,
                    objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id=0
                )

model_4 = xgb.XGBRegressor(
                    n_estimators=2000,
                    objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id=0
                )

model_5 = xgb.XGBRegressor(
                    n_estimators=2000,
                    objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id=0
                )

In [174]:
kfold = KFold(n_splits=2, shuffle=True)

random_search_1 = RandomizedSearchCV(model_1, param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

start = time.time()
random_search_1.fit(generated_train_x, generated_train_1)


print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter))

RandomizedSearchCV took 7289.08 seconds for 150 candidates parameter settings.


In [175]:
kfold = KFold(n_splits=2, shuffle=True)

random_search_2 = RandomizedSearchCV(model_2, param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

start = time.time()
random_search_2.fit(generated_train_x, generated_train_2)


print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter))

RandomizedSearchCV took 7411.21 seconds for 150 candidates parameter settings.


In [176]:
kfold = KFold(n_splits=2, shuffle=True)

random_search_3 = RandomizedSearchCV(model_3, param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

start = time.time()
random_search_3.fit(generated_train_x, generated_train_3)


print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter))

RandomizedSearchCV took 5736.81 seconds for 150 candidates parameter settings.


In [177]:
kfold = KFold(n_splits=2, shuffle=True)

random_search_4 = RandomizedSearchCV(model_4, param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

start = time.time()
random_search_4.fit(generated_train_x, generated_train_4)


print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter))

RandomizedSearchCV took 6394.66 seconds for 150 candidates parameter settings.


In [178]:
kfold = KFold(n_splits=2, shuffle=True)

random_search_5 = RandomizedSearchCV(model_5, param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

start = time.time()
random_search_5.fit(generated_train_x, generated_train_5)


print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter))

RandomizedSearchCV took 6826.02 seconds for 150 candidates parameter settings.


In [179]:
best_regressor_1 = random_search_1.best_estimator_

print(best_regressor_1.get_params())

{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.75, 'enable_categorical': False, 'gamma': 0, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.00999999978, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 5, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 2000, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1.25, 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'lambda': 1.25, 'eta': 0.01, 'alpha': 0.0}


In [180]:
best_regressor_2 = random_search_2.best_estimator_

print(best_regressor_2.get_params())

{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'enable_categorical': False, 'gamma': 0, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.00999999978, 'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 21, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 2000, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'lambda': 1, 'eta': 0.01, 'alpha': 0.0}


In [181]:
best_regressor_3 = random_search_3.best_estimator_

print(best_regressor_3.get_params())

{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.75, 'enable_categorical': False, 'gamma': 0, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.00999999978, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 2000, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1.25, 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'lambda': 1.25, 'eta': 0.01, 'alpha': 0.0}


In [182]:
best_regressor_4 = random_search_4.best_estimator_

print(best_regressor_4.get_params())

{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'enable_categorical': False, 'gamma': 0, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.00999999978, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 11, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 2000, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0.25, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.75, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'lambda': 1, 'eta': 0.01, 'alpha': 0.25}


In [183]:
best_regressor_5 = random_search_5.best_estimator_

print(best_regressor_5.get_params())

{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.75, 'enable_categorical': False, 'gamma': 0, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.00999999978, 'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 2000, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0.25, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'lambda': 1, 'eta': 0.01, 'alpha': 0.25}


In [184]:
pred_y_1 = best_regressor_1.predict(generated_test_x)
mse = mean_squared_error(generated_test_1, pred_y_1)
mae = mean_absolute_error(generated_test_1, pred_y_1)
print("Latent Feature 1")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)


pred_y_2 = best_regressor_2.predict(generated_test_x)
mse = mean_squared_error(generated_test_2, pred_y_2)
mae = mean_absolute_error(generated_test_2, pred_y_2)
print("\nLatent Feature 2")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)


pred_y_3 = best_regressor_3.predict(generated_test_x)
mse = mean_squared_error(generated_test_3, pred_y_3)
mae = mean_absolute_error(generated_test_3, pred_y_3)
print("\nLatent Feature 3")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

pred_y_4 = best_regressor_4.predict(generated_test_x)
mse = mean_squared_error(generated_test_4, pred_y_4)
mae = mean_absolute_error(generated_test_4, pred_y_4)
print("\nLatent Feature 4")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

pred_y_5 = best_regressor_5.predict(generated_test_x)
mse = mean_squared_error(generated_test_5, pred_y_5)
mae = mean_absolute_error(generated_test_5, pred_y_5)
print("\nLatent Feature 5")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

Latent Feature 1
MSE: 0.18
MAE: 0.33

Latent Feature 2
MSE: 0.10
MAE: 0.25

Latent Feature 3
MSE: 1.34
MAE: 0.99

Latent Feature 4
MSE: 0.95
MAE: 0.90

Latent Feature 5
MSE: 0.09
MAE: 0.25


In [185]:
index = 0

row=np.asarray([generated_test_x.iloc[index]])

y_1 = best_regressor_1.predict(row)
y_2 = best_regressor_2.predict(row)
y_3 = best_regressor_3.predict(row)
y_4 = best_regressor_4.predict(row)
y_5 = best_regressor_5.predict(row)

In [186]:
sample = np.asarray([[y_1[0], y_2[0], y_3[0], y_4[0], y_5[0]]])
og_coords = generated_test_x.iloc[index]
print(og_coords)
print(sample)

EMOTION_P    0.300917
EMOTION_A    0.399963
EMOTION_D    0.594633
Name: 5, dtype: float32
[[-0.04274109  0.08620528 -0.1836489  -0.08083361  0.26941812]]


In [187]:
regen = autoencoder.decoder.predict(sample)
print(regen)

[[ 0.53582144  0.43297517  0.36318842  0.5842653   0.24009949  0.34781405
   0.28055242  0.38740104  0.23633726  0.22389477  0.4874953   0.1107111
   0.09811359  0.04109593  0.20605248 -0.10380722 -0.1132756  -0.08900352
  -0.13064082 -0.1087364   0.01292793  0.01732282  0.02132782  0.0204859
   0.01254203]]


In [188]:
scaled_regen = scaler.transform(regen)

generated_coordinates = (
    model_p.predict(scaled_regen),
    model_a.predict(scaled_regen),
    model_d.predict(scaled_regen)
)


print('Real: %s' % np.asarray(og_coords))
print('Generated: %s' % np.asarray(generated_coordinates).transpose()[0])

Real: [0.30091733 0.39996335 0.594633  ]
Generated: [0.10747234 0.28109705 0.21227694]


### Test

In [191]:
# 1st - Generate each lma feature for each X
generated_features = []

pred_feature = best_regressor_1.predict(generated_test_x)
generated_features.append(pred_feature)

pred_feature = best_regressor_2.predict(generated_test_x)
generated_features.append(pred_feature)

pred_feature = best_regressor_3.predict(generated_test_x)
generated_features.append(pred_feature)

pred_feature = best_regressor_4.predict(generated_test_x)
generated_features.append(pred_feature)

pred_feature = best_regressor_5.predict(generated_test_x)
generated_features.append(pred_feature)

generated_features = np.asarray(generated_features).transpose()

print(generated_features.shape)

(7201, 5)


In [192]:
# 2nd - Use Decoder to get LMA features
generated_lma = autoencoder.decoder.predict(generated_features)
print(generated_lma.shape)

scaled_generated_lma = scaler.transform(generated_lma)

(7201, 25)


In [193]:
# 3rd - Get Predicted Emotions
gen_emotions_p = model_p.predict(scaled_generated_lma)
gen_emotions_a = model_a.predict(scaled_generated_lma)
gen_emotions_d = model_d.predict(scaled_generated_lma)

rows = []
for i in range(len(scaled_generated_lma)):
    rows.append([gen_emotions_p[i], gen_emotions_a[i], gen_emotions_d[i]])

gen_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

gen_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.107472,0.281097,0.212277
1,0.196899,-0.131000,0.026693
2,0.093880,0.454136,0.259302
3,0.236589,-0.011891,0.058046
4,0.197668,-0.098458,0.032507


In [194]:
# 4th - Get MAE
import random

mae_errors = mean_absolute_error(test_emotions, gen_emotions, multioutput='raw_values')
mse_errors = mean_squared_error(test_emotions, gen_emotions, multioutput='raw_values')

features = ["PLEASURE", "AROUSAL", "DOMINANCE"
         ]

print("Overall MAE: " + str(mean_absolute_error(test_emotions, gen_emotions)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print()
    
for i in range(30):
    row = random.randint(0, len(test_emotions))

    print("Real: " + str([test_emotions_OG.iloc[row,0], test_emotions_OG.iloc[row,1], test_emotions_OG.iloc[row,2]]))
    print("Predicted: " + str([test_emotions.iloc[row,0], test_emotions.iloc[row,1], test_emotions.iloc[row,2]]))
    print("Generated: " + str([gen_emotions.iloc[row,0], gen_emotions.iloc[row,1], gen_emotions.iloc[row,2]]))
    print()

Overall MAE: 0.27214193

==PLEASURE==
MSE: 0.05287
MAE: 0.19422

==AROUSAL==
MSE: 0.21352
MAE: 0.40835

==DOMINANCE==
MSE: 0.06444
MAE: 0.21386

Real: [0.05, -0.05, 0.0]
Predicted: [0.050158333, -0.051505435, -0.004425468]
Generated: [0.22378723, -0.24974051, -0.1482873]

Real: [0.1, -0.75, -0.25]
Predicted: [0.10005977, -0.7482347, -0.25138375]
Generated: [0.15950416, -0.48422387, -0.15814184]

Real: [-0.1, -0.6, -0.15]
Predicted: [-0.09410743, -0.56806827, -0.13370807]
Generated: [0.10262783, 0.27089253, 0.1503368]

Real: [0.05, -0.05, 0.0]
Predicted: [0.050600104, -0.048232384, -0.0007531466]
Generated: [0.1557518, -0.27808547, -0.06786705]

Real: [0.1, 0.6, 0.4]
Predicted: [0.10017814, 0.5982219, 0.40379992]
Generated: [0.12245319, -0.0973924, 0.12101584]

Real: [0.6, 0.4, 0.1]
Predicted: [0.6032445, 0.40349478, 0.09395698]
Generated: [0.18910787, -0.0049485196, 0.13966668]

Real: [0.1, 0.6, 0.4]
Predicted: [0.100123085, 0.59902155, 0.40392357]
Generated: [0.20612247, -0.0835139, 0

In [62]:
best_regressor_1.save_model("models/bandai/5frame/ae/bandai_pad2l1_model.json")
best_regressor_2.save_model("models/bandai/5frame/ae/bandai_pad2l2_model.json")
best_regressor_3.save_model("models/bandai/5frame/ae/bandai_pad2l3_model.json")
best_regressor_4.save_model("models/bandai/5frame/ae/bandai_pad2l4_model.json")
best_regressor_5.save_model("models/bandai/5frame/ae/bandai_pad2l5_model.json")